In [3]:
from scipy.io import arff # import arff files
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
from matplotlib import rcParams # figure size
from sklearn.preprocessing import LabelEncoder # wrangle strings to floats

In [ ]:
# importing training data
data = arff.loadarff('/content/training_subsetD.arff')
df = pd.DataFrame(data[0])
str_df = df.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
df = str_df
df.shape

In [ ]:
# importing test data
data = arff.loadarff('/content/testingD.arff')
dftest = pd.DataFrame(data[0])
str_dftest = dftest.select_dtypes([np.object])
str_dftest = str_dftest.stack().str.decode('utf-8').unstack()
str_dftest.head()
dftest = str_dftest
dftest.shape

In [ ]:
#Merging data unique strings + merging these + return unique strings
dflist = np.ndarray.tolist(np.unique(df))
dftestlist = np.ndarray.tolist(np.unique(dftest))
onelist = dflist + dftestlist
len(np.unique(onelist))

In [111]:
#Encode the unique strings for processing in scikit learn
le = LabelEncoder()
le.fit(onelist)
df = np.array([le.transform(i) for i in df.values])
dftest = np.array([le.transform(i) for i in dftest.values])

In [ ]:
#Import the DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

#Assign data labels
train_features = df[:,:-1]
test_features = dftest[:,:-1]
train_targets = df[:,-1]
test_targets = dftest[:,-1]

#Train the model
tree = DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_targets)

#Predict the classes of new, unseen data
#prediction = tree.predict(test_features)

#Check the accuracy
print("The prediction accuracy is: ",tree.score(test_features,test_targets)*100,"%")
